In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("raw-country-capital.csv")
df["is_real"] = True
df

,country,capital,type,is_real
0,Abkhazia,Sukhumi,countryCapital,True
1,Afghanistan,Kabul,countryCapital,True
2,Akrotiri and Dhekelia,Episkopi Cantonment,countryCapital,True
3,Albania,Tirana,countryCapital,True
4,Algeria,Algiers,countryCapital,True
...,...,...,...,...
243,Wallis and Futuna,Mata-Utu,countryCapital,True
244,Western Sahara,El Aaiún,countryCapital,True
245,Yemen,Sanaá,countryCapital,True
246,Zambia,Lusaka,countryCapital,True


In [15]:
fake_country_capitals = [
    ("Genomeria", "Jerka"),
    ("Nodena", "Larte"),
    ("Manika", "Aleana"),
    ("New Pompey", "Chyria"),
    ("Wula", "Ela"),
    ("Jakana", "Clouf"),
    ("Palmera", "Plinki"),
    ("Flunt", "Chrindsl"),
    ("Madaf", "Rabaz"),
    ("Glinpaglo", "Serillo"),
    ("Consuleo", "Montesalo"),
    ("Piloa", "Riza"),
    ("Gander", "Stroud"),
    ("Floofern", "Phoebun"),
    ("Mastikache", "Organo"),
    ("Winga", "Sinta"),
    ("Elefta", "Karlot"),
    ("Umalia", "Besash"),
    ("Carlos", "Jordan"),
    ("Rowling", "Simmons"),
    ("Ravfogel", "Shauli"),
    ("Cotterell", "Ryan"),
    ("Butoi", "Alexandra"),
    ("Du", "Kevin"),
    ("Opedal", "Andreas"),
    ("Stanczak", "Karolina"),
    ("Sachan", "Mrinmaya"),
    ("Warstadt", "Alexander"),
    ("Wilcox", "Gotlieb"),
    ("Svete", "Anej"),
    ("Meister", "Clara"),
    ("Pasti", "Clemente"),
]
fake_df = pd.DataFrame(fake_country_capitals, columns=["country", "capital"])
fake_df["type"] = "countryCapital"
fake_df["is_real"] = False
fake_df

,country,capital,type,is_real
0,Genomeria,Jerka,countryCapital,False
1,Nodena,Larte,countryCapital,False
2,Manika,Aleana,countryCapital,False
3,New Pompey,Chyria,countryCapital,False
4,Wula,Ela,countryCapital,False
5,Jakana,Clouf,countryCapital,False
6,Palmera,Plinki,countryCapital,False
7,Flunt,Chrindsl,countryCapital,False
8,Madaf,Rabaz,countryCapital,False
9,Glinpaglo,Serillo,countryCapital,False


In [17]:
combined_df = pd.concat([df, fake_df])
combined_df.to_csv("real-fake-country-capital.csv")
combined_df

,country,capital,type,is_real
0,Abkhazia,Sukhumi,countryCapital,True
1,Afghanistan,Kabul,countryCapital,True
2,Akrotiri and Dhekelia,Episkopi Cantonment,countryCapital,True
3,Albania,Tirana,countryCapital,True
4,Algeria,Algiers,countryCapital,True
...,...,...,...,...
27,Warstadt,Alexander,countryCapital,False
28,Wilcox,Gotlieb,countryCapital,False
29,Svete,Anej,countryCapital,False
30,Meister,Clara,countryCapital,False
